In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import os

from predict_policing_agent import PredictivePolicingAgent, PredictivePolicingAgentParams
from predict_policing_env import PredictivePolicingEnv
import predict_policing_env
from predict_hotspot_seppR_model import predict_hotspot_SEPP_R_model, get_bogota_mask
from predict_policing_simulation import *
import predict_policing_fairness_metrics 
import time
import multiprocessing as mp

import sys
sys.path.insert(0, os.getcwd() + '/../')
import vanilla_monte_carlo

In [2]:
crime_data_path = "data/real_bogota_victim_data_v21_ts2555.csv"
model_param_path = "output/version_21/learned_parameters/learned_parameters_thinning_FALSE.csv"
N_related_days = 190 # non thinned
cells_mask_path = "metadata/bogota_mask_1.csv"
gt_50hotspot_num_path = "output/version_21/gt_int_hotspot_num.csv"
gt_50hotspot_num = pd.read_csv(gt_50hotspot_num_path, index_col=0)
gt_real_hot_spots_path = "output/version_21/gt_real_hot_spot_cells.pkl"
start_ts = 2000
num_steps = 400

init_params = {'crime_data_path': crime_data_path,
                'model_param_path': model_param_path,
                'cells_mask_path': cells_mask_path,
                'gt_real_hot_spots_path': gt_real_hot_spots_path,
                'start_ts': start_ts,
                'gt_n_hot_spots': gt_50hotspot_num,
                'n_related_days': N_related_days,
                'n_time_steps': num_steps}

potential_hot_spot_discover_rate = [1.0, 0.95, 0.9, 0.85, 0.8]
potential_non_hot_spot_discover_rate = [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2]
potential_n_hot_spots = [50]
potential_hot_spot_effect_size = [1, 2, 3]
potential_rel_num_gap_threshold = [0.30]
potential_fairness_window_size = [30]

# get all possible combinations of parameters
all_params = [potential_hot_spot_discover_rate, potential_non_hot_spot_discover_rate, potential_n_hot_spots,
                potential_hot_spot_effect_size, potential_rel_num_gap_threshold,
                potential_fairness_window_size]
all_params = np.array(np.meshgrid(*all_params)).T.reshape(-1, len(all_params))

all_configs = []
for i in range(len(all_params)):
    hot_spot_discover_rate = all_params[i][0]
    non_hot_spot_discover_rate = all_params[i][1]
    n_hot_spots = int(all_params[i][2])
    hot_spot_effect_size = int(all_params[i][3])
    rel_num_gap_threshold = all_params[i][4]
    fairness_window_size = int(all_params[i][5]) 
    average_relative_num_gap_requirement = predict_policing_fairness_metrics.AverageRelativeNumberGap(rel_num_gap_threshold, 
                                                                                                        fairness_window_size)
    config = PredictPolicingConfiguration(hot_spot_discover_rate, non_hot_spot_discover_rate, n_hot_spots,
                                          hot_spot_effect_size, average_relative_num_gap_requirement)
    all_configs.append([i, config])

In [3]:
len(all_configs)

105

In [4]:
# create a vanilla testing object
import multiprocessing as mp
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[:1], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results_test = vanilla_test.run_testing(n_threads=8, t_limit=60)

Total time used: 84.70347595214844 seconds


In [5]:
# create a vanilla testing object
import multiprocessing as mp
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[0:5], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results0 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/00-05_w_utility.pkl', 'wb') as f:
        pickle.dump(results0, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[5:10], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results1 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/05-10_w_utility.pkl', 'wb') as f:
        pickle.dump(results1, f)


Total time used: 20102.19899535179 seconds
Total time used: 18764.706334352493 seconds


In [11]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[10:15], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results2 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/10-15_w_utility.pkl', 'wb') as f:
        pickle.dump(results2, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[15:20], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results3 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/15-20_w_utility.pkl', 'wb') as f:
        pickle.dump(results3, f)

Total time used: 16807.7390897274 seconds
Total time used: 16552.780429124832 seconds


In [12]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[20:25], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results4 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/20-25_w_utility.pkl', 'wb') as f:
        pickle.dump(results4, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[25:30], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results5 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/25-30_w_utility.pkl', 'wb') as f:
        pickle.dump(results5, f)

Total time used: 20340.40075802803 seconds
Total time used: 18773.915551185608 seconds


In [13]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[30:35], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results6 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/30-35_w_utility.pkl', 'wb') as f:
        pickle.dump(results6, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[35:40], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results7 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/35-40_w_utility.pkl', 'wb') as f:
        pickle.dump(results7, f)

Total time used: 14522.270394563675 seconds
Total time used: 20370.6916615963 seconds


In [14]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[40:45], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results8 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/40-45_w_utility.pkl', 'wb') as f:
        pickle.dump(results8, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[45:50], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results9 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/45-50_w_utility.pkl', 'wb') as f:
        pickle.dump(results9, f)

Total time used: 18724.107235193253 seconds
Total time used: 21303.8163254261 seconds


In [15]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[50:55], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results10 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/50-55_w_utility.pkl', 'wb') as f:
        pickle.dump(results10, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[55:60], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results11 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/55-60_w_utility.pkl', 'wb') as f:
        pickle.dump(results11, f)

Total time used: 21561.779905319214 seconds
Total time used: 22803.251734256744 seconds


In [16]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[60:65], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results12 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/60-65_w_utility.pkl', 'wb') as f:
        pickle.dump(results12, f)
    
    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[65:70], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results13 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/65-70_w_utility.pkl', 'wb') as f:
        pickle.dump(results13, f)

Total time used: 17117.24264907837 seconds


In [5]:
if __name__ == '__main__':
    stop_criteria = (MaxUnfairnessStopCriteria, 0.02)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[65:70], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results13 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/65-70_w_utility.pkl', 'wb') as f:
        pickle.dump(results13, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[70:75], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results14 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/70-75_w_utility.pkl', 'wb') as f:
        pickle.dump(results14, f)

    vanilla_test = vanilla_monte_carlo.VanillaMonteCarloTestingSingle(all_configs[75:80], init_params, PredictivePolicingMonteCarloSimulation, stop_criteria)
    results15 = vanilla_test.run_testing(n_threads=8, t_limit=4800)
    with open('simulation_results/75-80_w_utility.pkl', 'wb') as f:
        pickle.dump(results15, f)

Total time used: 24567.150334358215 seconds
Total time used: 23557.38680911064 seconds
Total time used: 23493.958874940872 seconds
